In [64]:
from xml.etree import ElementTree as ET
from xml.dom import minidom
import copy

In [313]:
def add_comment(elem, comment):
    res = copy.copy(elem)
    res.append(ET.Comment(comment))
    return res

def add_subelement(dom, tag, *, text='', attrs=dict()):
    if type(dom) == type(minidom.Document()):
        res = add_subelement_minidom(dom, tag, text, attrs)
    elif type(dom) == type(ET.Element('')):
        res = add_subelement_et(dom, tag, text, attrs)
    return res

def add_subelement_et(dom, tag, text, attrs):
    res = copy.copy(dom)
    child = ET.SubElement(res, tag, attrib=attrs)
    if text != '':
        child.text = text
    return res

def add_subelement_minidom(dom, tag, text, attrs):
    res = copy.copy(dom)
    child = res.createElement(tag)
    if text != '':
        child.appendChild(dom.createTextNode(text))
    if attrs is not None:
        for  k, v in attrs.items():
            attr = dom.createAttribute(k)
            attr.value = v
            child.setAttributeNode(attr)
    res.appendChild(child)
    return res

def get_prettified(elem):
    if type(elem) == type(ET.Element('')):
        et_str = ET.tostring(elem, 'unicode')
        print(et_str)
        elem = ET.fromstring(et_str)
    return elem.toprettyxml(indent='\t', encoding='unicode').decode()

In [314]:
from datetime import datetime, timezone, timedelta

In [315]:
def now_formatted():
    return datetime.strftime(
        datetime.now(timezone(timedelta(hours=+9), 'JST')),
        '%a, %d %b %Y %H:%M:%S %z'
    )

In [321]:
def make_channel_elem(
    title,
    link,
    desc,
    build_date,
    lang,
    summary,
    authors,
    explicit,
    image_attrs,
    type,
    category
):
    '''
    Make xml item that contains channel information
    Argument descriptions are copied from https://help.apple.com/itc/podcasts_connect/#/itcb54353390.
    See https://help.apple.com/itc/podcasts_connect/#/itcb54353390 for more detail.
    
    parameters
    ----------
    title:
        (Required) The show title.
        It's important to have a clear, concise name for your podcast. Make your title specific. A show titled Our Community Bulletin is too vague to attract many subscribers, no matter how compelling the content.
        Pay close attention to the title as Apple Podcasts uses this field for search.
        If you include a long list of keywords in an attempt to game podcast search, your show may be removed from the Apple directory.
        
    desc:
        (Required) The show description.
        Where description is text containing one or more sentences describing your podcast to potential listeners. The maximum amount of text allowed for this tag is 4000 bytes.

        To include links in your description or rich HTML, adhere to the following technical guidelines: enclose all portions of your XML that contain embedded HTML in a CDATA section to prevent formatting issues, and to ensure proper link functionality. For example:
        ```xml
        <![CDATA[
            <a href="http://www.apple.com">Apple</a>
        ]]>
        ```
    lang:
        (Required) The language spoken on the show.
        Because Apple Podcasts is available in territories around the world, it is critical to specify the language of a podcast. Apple Podcasts only supports values from the ISO 639 list (two-letter language codes, with some possible modifiers, such as "en-us").

        Invalid language codes will cause your feed to fail Apple validation.
    category:
        (Required) The show category information. For a complete list of categories and subcategories, see Apple Podcast categories.
        Select the category that best reflects the content of your show. If available, you can also define a subcategory.

        Although you can specify more than one category and subcategory in your RSS feed, Apple Podcasts only recognizes the first category and subcategory.

        When specifying categories and subcategories, be sure to properly escape ampersands. For example:

        Single category:
        ```xml
        <itunes:category text="History" />
        ```
        Category with ampersand:
        ```xml
        <itunes:category text="Kids &amp; Family" />
        ```
        Category with subcategory:

        ```xml
        <itunes:category text="Society &amp; Culture">
            <itunes:category text="Documentary" />
        </itunes:category>
        ```
        Multiple categories:

        ```xml
        <itunes:category text="Society &amp; Culture">
            <itunes:category text="Documentary" />
        </itunes:category>
        <itunes:category text="Health">
            <itunes:category text="Mental Health" />
        </itunes:category>
        ```
        
        explicit:
            (Required) The podcast parental advisory information.

            The explicit value can be one of the following:

            True. If you specify true, indicating the presence of explicit content, Apple Podcasts displays an Explicit parental advisory graphic for your podcast.

            Podcasts containing explicit material aren't available in some Apple Podcasts territories.

            False. If you specify false, indicating that your podcast doesn't contain explicit language or adult content, Apple Podcasts displays a Clean parental advisory graphic for your podcast.
        
        authors:
            (Recommended) The group responsible for creating the show.

            Show author most often refers to the parent company or network of a podcast, but it can also be used to identify the host(s) if none exists.

            Author information is especially useful if a company or organization publishes multiple podcasts.
            
        link:
            (Recommended) The website associated with a podcast.
            Typically a home page for a podcast or a dedicated portion of a larger website. For example:

            ```xml
            <link>
            http://www.mypodcast.com
            </link>
            ```
            or

            ```xml
            <link>
            http://www.mediacompany.com/podcast
            </link>
            ```
            
        owner: (not implemented)
            (Recommended) The podcast owner contact information.

            Include the email address of the owner in a nested <itunes:email> tag and the name of the owner in a nested <itunes:name> tag.

            Note: The <itunes:owner> tag information is for administrative communication about the podcast and isn’t displayed in Apple Podcasts. Please make sure the email address is active and monitored.
            
        type:
            (Recommended) The type of show.
            If your show is Serial you must use this tag.

            Its values can be one of the following:

            Episodic (default). Specify episodic when episodes are intended to be consumed without any specific order. Apple Podcasts will present newest episodes first and display the publish date (required) of each episode. If organized into seasons, the newest season will be presented first - otherwise, episodes will be grouped by year published, newest first.

            For new subscribers, Apple Podcasts adds the newest, most recent episode in their Library.

            Serial. Specify serial when episodes are intended to be consumed in sequential order. Apple Podcasts will present the oldest episodes first and display the episode numbers (required) of each episode. If organized into seasons, the newest season will be presented first and <itunes:episode> numbers must be given for each episode.

            For new subscribers, Apple Podcasts adds the first episode to their Library, or the entire current season if using seasons.
    '''
    res = ET.Element('channel')
    res = add_subelement(res, 'title', text=title)
    res = add_subelement(res, 'link', text=link)
    res = add_subelement(res, 'description', text=desc)
    res = add_subelement(res, 'lastBuildDate', text=build_date)
    res = add_subelement(res, 'language', text=lang)
    res = add_subelement(res, 'itunes:new-feed-url', text=f'{link}/feed')
    res = add_subelement(res, 'itunes:summary', text=f'![CDATA[{summary}]]')
    for author in authors:
        res = add_subelement(res, 'itunes:author', text=author)
    res = add_subelement(res, 'itunes:explicit', text=explicit)
    res = add_subelement(res, 'itunes:image', attrs=image_attrs)
    res = add_subelement(res, 'itunes:type', text=type)
    res = add_subelement(res, 'itunes:category', text=category)

    return res

In [320]:
def make_image_elem(title, link, image_url, size):
    '''
    Make xml item of the channel/episode artwork
    Argument descriptions are copied from https://help.apple.com/itc/podcasts_connect/#/itcb54353390.
    See https://help.apple.com/itc/podcasts_connect/#/itcb54353390 for more detail.
    
    parameters
    ----------
    title, link:
        see make_channel_elem()
        
    url, width, height:
        (For channel)
        Specify your show artwork by providing a URL linking to it.
        Depending on their device, subscribers see your podcast artwork in varying sizes. Therefore, make sure your design is effective at both its original size and at thumbnail size. You should include a show title, brand, or source name as part of your podcast artwork. Here are additional marketing best practices. For examples of podcast artwork, see the Top Podcasts chart. To avoid technical issues when you update your podcast artwork, be sure to:

        Change the artwork file name and URL at the same time

        Verify the web server hosting your artwork allows HTTP head requests

        Artwork must be a minimum size of 1400 x 1400 pixels and a maximum size of 3000 x 3000 pixels, in JPEG or PNG format, 72 dpi, with appropriate file extensions (.jpg, .png), and in the RGB colorspace. These requirements are different from the standard RSS image tag specifications.

        Make sure the file type in the URL matches the actual file type of the image file.
        
        (For episode)

        The episode artwork.

        You should use this tag when you have a high quality, episode-specific image you would like listeners to see.

        Specify your episode artwork using the href attribute in the <itunes:image> tag. RSS Feed Sample.

        Depending on their device, listeners see your episode artwork in varying sizes. Therefore, make sure your design is effective at both its original size and at thumbnail size. You should include a title, brand, or source name as part of your episode artwork. To avoid technical issues when you update your episode artwork, be sure to:

        Change the artwork file name and URL at the same time

        Verify the web server hosting your artwork allows HTTP head requests

        Artwork must be a minimum size of 1400 x 1400 pixels and a maximum size of 3000 x 3000 pixels, in JPEG or PNG format, 72 dpi, with appropriate file extensions (.jpg, .png), and in the RGB colorspace. These requirements are different from the standard RSS image tag specifications.

        Make sure the file type in the URL matches the actual file type of the image file.
    '''
    res = ET.Element('image')
    res = add_subelement(res, 'title', text=title)
    res = add_subelement(res, 'link', text=link)
    res = add_subelement(res, 'url', text=image_url)
    res = add_subelement(res, 'width', text=size)
    res = add_subelement(res, 'height', text=size)
    return res

In [329]:
def make_episode_item(
    title,
    link,
    pub_date,
    guid,
    desc,
    content,
    enclosure_attrs,
    subtitle,
    summary,
    authors,
    image_url,
    duration
):
    '''
    Make xml item that contains episode information
    Argument descriptions are copied from https://help.apple.com/itc/podcasts_connect/#/itcb54353390.
    See https://help.apple.com/itc/podcasts_connect/#/itcb54353390 for more detail.
    
    parameters
    ----------
    title:
        (Required) An episode title.

        title is a string containing a clear, concise name for your episode.

        Don’t specify the episode number or season number in this tag. Instead, specify those details in the appropriate tags ( <itunes:episode>, <itunes:season>). Also, don’t repeat the title of your show within your episode title.

        Separating episode and season number from the title makes it possible for Apple to easily index and order content from all shows.
    enclosure_attrs:
        (Required) The episode content, file size, and file type information.

        The <enclosure> tag has three attributes: URL, length, and type:

        URL. The URL attribute points to your podcast media file. The file extension specified within the URL attribute determines whether or not content appears in the podcast directory. Supported file formats include M4A, MP3, MOV, MP4, M4V, and PDF.

        Length. The length attribute is the file size in bytes. You can find this information in the properties of your podcast file (on a Mac, choose File > Get Info and refer to the size field).

        Type. The type attribute provides the correct category for the type of file you are using. The type values for the supported file formats are: audio/x-m4a, audio/mpeg, video/quicktime, video/mp4, video/x-m4v, and application/pdf.

        For example:
        ```xml
        <enclosure
        url="http://mypodcast.com/episode001.mp3" 
        length="5650889"
        type="audio/mpeg
        />
        ```
    guid:
        (Required) The episode’s globally unique identifier (GUID) If you uploaded subscriber audio in Apple Podcasts Connect and need to link it to an episode in your RSS feed, you can use the Apple Podcasts Episode ID in the GUID tag. Learn more about how to set up your show for a subscription.

        It is very important that each episode have a unique GUID and that it never changes, even if an episode’s metadata, like title or enclosure URL, do change.

        Globally unique identifiers (GUID) are case-sensitive strings. If a GUID is not provided, an episode’s enclosure URL will be used instead. If a GUID is not provided, make sure that an episode’s enclosure URL is unique and never changes.

        Failing to comply with these guidelines may result in duplicate episodes being shown to listeners, inaccurate data in Analytics, and can cause issues with your podcasts’s listing and chart placement in Apple Podcasts.
    
    pub_date:
        (Required) The date and time when an episode was released.
        Format the date using the RFC 2822 specifications. For example: Wed, 15 Jun 2019 19:00:00 GMT.
        
    desc:
        (Recommended) An episode description.

        description is text containing one or more sentences describing your episode to potential listeners. You can specify up to 4000 bytes. You can use rich text formatting and some HTML (<p>, <ol>, <ul>, <li>, <a>) if wrapped in the <CDATA> tag.

        To include links in your description or rich HTML, adhere to the following technical guidelines: enclose all portions of your XML that contain embedded HTML in a CDATA section to prevent formatting issues, and to ensure proper link functionality. For example:

        ```xml
        <![CDATA[
            <a href="http://www.apple.com">Apple</a>
        ]]>
        ```
        
    duration:
        (Recommended) The duration of an episode.

        Different duration formats are accepted however it is recommended to convert the length of the episode into seconds.
    
    link:
        (Recommended) An episode link URL.
        This is used when an episode has a corresponding webpage. For example:
        ```xml
        <link>
        http://www.mypodcast.com/episode-one.html
        </link>
        ```
        
    explicit:
        (Recommended) The episode parental advisory information.

        Where the explicit value can be one of the following:

        true. If you specify true, indicating the presence of explicit content, Apple Podcasts displays an Explicit parental advisory graphic for your episode.

        Episodes containing explicit material aren’t available in some Apple Podcasts territories.

        false. If you specify false, indicating that the episode does not contain explicit language or adult content, Apple Podcasts displays a Clean parental advisory graphic for your episode.
    '''
    res = ET.Element('item')
    res = add_subelement(res, 'title', text=title)
    res = add_subelement(res, 'link', text=link)
    res = add_subelement(res, 'pubDate', text=pub_date)
    res = add_subelement(res, 'guid', text=guid)
    res = add_subelement(res, 'description', text=desc)
    res = add_subelement(res, 'content:encoded', text=content)
    
    res = add_subelement(res, 'enclosure', attrs=enclosure_attrs)
    res = add_subelement(res, 'itunes:subtitle', text=subtitle)
    res = add_subelement(res, 'itunes:summary', text=summary)
    for author in authors:
        res = add_subelement(res, 'itunes:author', text=author)
    res = add_subelement(res, 'itunes:image', text=image_url)
    res = add_subelement(res, 'itunes:duration', text=duration)
    return res

In [340]:
def make_channel():
    channel = make_channel_elem(
        title='フットボールしぶ！',
        link='https://football-shibu.web.app/',
        desc='サッカーについて話します',
        build_date=now_formatted(),
        lang='ja',
        summary='サッカーについて話します',
        authors=['Gota Shirato', 'Yuki Yuda'],
        explicit='clean',
        image_attrs={'href': 'https://sports-con.xyz/wp-content/uploads/2022/02/football-shiv.png'},
        type='episodic',
        category='Sports'
    )
    image = make_image_elem('フットボールしぶ！', 'https://sports-con.xyz/wp-content/uploads/2022/02/football-shiv.png', 'https://football-shibu.web.app/', '32')
    channel.append(image)
    
    episode = make_episode_item(
        title='#17 開幕直前!!J1順位予想',
        link='https://football-shibu.web.app/',
        pub_date=now_formatted(),
        guid='https://football-shibu.web.app/', #same as link?
        desc='開幕直前!!J1順位予想',
        content='開幕直前!!J1順位予想',
        enclosure_attrs={
            'url': 'https://sports-con.xyz/wp-content/uploads/2022/02/football-17.mp3',
            'length': '39828750',
            'type': 'audio/mpeg'},
        subtitle='#17 開幕直前!!J1順位予想',
        summary='開幕直前!!J1順位予想',
        authors=['Gota Shirato', 'Yuki Yuda'],
        image_url='https://sports-con.xyz/wp-content/uploads/2022/02/icon-football-17.jpg',
        duration='01:22:58'
    )
    channel.append(episode)
    
    return channel

In [341]:
rss_attrs = {
    'version':'2.0',
    'xmlns:itunes': 'http://www.itunes.com/dtds/podcast-1.0.dtd',
    'xmlns:content': 'http://purl.org/rss/1.0/modules/content/'
}

In [342]:
dom = ET.Element('rss', attrib=rss_attrs)
tree = add_subelement(dom, 'rss', attrs=rss_attrs)
channel = make_channel()
dom.append(channel)

In [343]:
ET.tostring(dom, encoding='unicode', method='xml')

'<rss version="2.0" xmlns:itunes="http://www.itunes.com/dtds/podcast-1.0.dtd" xmlns:content="http://purl.org/rss/1.0/modules/content/"><channel><title>フットボールしぶ！</title><link>https://football-shibu.web.app/</link><description>サッカーについて話します</description><lastBuildDate>Wed, 23 Feb 2022 01:17:01 +0900</lastBuildDate><language>ja</language><itunes:new-feed-url>https://football-shibu.web.app//feed</itunes:new-feed-url><itunes:summary>![CDATA[サッカーについて話します]]</itunes:summary><itunes:author>Gota Shirato</itunes:author><itunes:author>Yuki Yuda</itunes:author><itunes:explicit>clean</itunes:explicit><itunes:image href="https://sports-con.xyz/wp-content/uploads/2022/02/football-shiv.png" /><itunes:type>episodic</itunes:type><itunes:category>Sports</itunes:category><image><title>フットボールしぶ！</title><link>https://sports-con.xyz/wp-content/uploads/2022/02/football-shiv.png</link><url>https://football-shibu.web.app/</url><width>32</width><height>32</height></image><item><title>#17 開幕直前!!J1順位予想</title><link>

In [349]:
import os

In [355]:
def writexml(obj, path, encoding):
    with open(path, 'w') as f:
        obj.writexml(f, encoding='unicode', newl='\n', indent='', addindent='\t')

In [356]:
def get_now_str():
    return datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d-%H%M%S')

In [357]:
def save_feed(dom, rss_folder, encoding='unicode'):
    document = minidom.parseString(ET.tostring(dom, 'unicode', xml_declaration=True))
    writexml(document, os.path.join(rss_folder, 'feed.rss'), encoding=encoding)
    writexml(document, os.path.join(rss_folder, '.logs', f'{get_now_str()}.rss'), encoding=encoding)

In [358]:
save_feed(dom, '../rss')

In [346]:
import datetime

'20220222-193404'